In [ ]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Load dataset
df = pd.read_csv('../Datasets/CleanData.csv')
df_dep = pd.read_csv('../Datasets/Depression.csv')
df_anx = pd.read_csv('../Datasets/Anxiety.csv')
df_str = pd.read_csv('../Datasets/Stress.csv')

In [ ]:
# Dataset info
df.shape
df.columns
df.dtypes
df.isnull().sum()

In [ ]:
# Extract features and target
X = df.iloc[:, 0:7].values
Y = df.iloc[:, 7].values
X_dep = df_dep.iloc[:, :-1].values
Y_dep = df_dep.iloc[:, -1].values
X_anx = df_anx.iloc[:, :-1].values
Y_anx = df_anx.iloc[:, -1].values
X_str = df_str.iloc[:, :-1].values
Y_str = df_str.iloc[:, -1].values

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)
X_train_dep, X_test_dep, y_train_dep, y_test_dep = train_test_split(X_dep, Y_dep, test_size=0.3, random_state=1)
X_train_anx, X_test_anx, y_train_anx, y_test_anx = train_test_split(X_anx, Y_anx, test_size=0.3, random_state=1)
X_train_str, X_test_str, y_train_str, y_test_str = train_test_split(X_str, Y_str, test_size=0.3, random_state=1)

In [ ]:
# Define and evaluate KNN model
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
knn_acc = accuracy_score(y_test, y_pred)
knn_cls = classification_report(y_test, y_pred)
knn_acc
knn_cls

knn.fit(X_train_dep, y_train_dep)
y_pred_dep = knn.predict(X_test_dep)
knn_acc_dep = accuracy_score(y_test_dep, y_pred_dep)
knn_cls_dep = classification_report(y_test_dep, y_pred_dep)
knn_acc_dep
knn_cls_dep

In [ ]:
# Evaluate KNN model on Anxiety and Stress
y_pred_anx = knn.predict(X_test_anx)
knn_acc_anx = accuracy_score(y_test_anx, y_pred_anx)
knn_cls_anx = classification_report(y_test_anx, y_pred_anx)
knn_acc_anx
knn_cls_anx
y_pred_str = knn.predict(X_test_str)
knn_acc_str = accuracy_score(y_test_str, y_pred_str)
knn_cls_str = classification_report(y_test_str, y_pred_str)
knn_acc_str
knn_cls_str